In [555]:
import urllib, json
import pandas as pd
import re
from itertools import permutations
from pulp import *
import numpy as np
import random
from random import choice

from itertools import chain




In [556]:
base = pd.read_csv("combined_data/combined_database_pool.csv")

In [557]:
#dropping useless column
one =base.drop('Game Info', axis =1)

In [558]:
#dropping useless column
two = one.drop('Unnamed: 0', axis =1)

In [559]:
#dropping useless column
three= two.drop('TeamAbbrev', axis=1)


In [560]:
#dropping useless column
four = three.drop('Name + ID', axis=1)

In [561]:

player_arr = four.to_numpy()

In [642]:
qbs= four[four.Position == 'QB']
rbs= four[four.Position == 'RB'].drop('ID', axis =1)
wrs= four[four.Position == 'WR'].drop('ID', axis =1)
tes= four[four.Position == 'TE'].drop('ID', axis =1)
#flex= four[four.Position == 'WR'].drop('ID', axis =1)

#flex = four[four.Position == 'WR' or four[four.Position == 'Rb']
            
flex = four[(four.Position == 'WR') | (four.Position == 'RB')]
defs = four[four.Position == 'DST'].drop('ID', axis =1)

qbs = qbs.drop('ID', axis = 1)

#had to drop ID off Flex DB again
flex_no_id =flex.drop('ID', axis =1)

flex_no_id

,Position,Name,Roster Position,Salary,AvgPointsPerGame
0,RB,Dalvin Cook,RB/FLEX,9400,26.94
1,RB,Christian McCaffrey,RB/FLEX,9200,30.13
2,RB,Derrick Henry,RB/FLEX,8700,20.74
3,RB,Aaron Jones,RB/FLEX,7600,19.93
4,RB,James Robinson,RB/FLEX,7500,19.73
...,...,...,...,...,...
110,WR,Emmanuel Sanders,WR/FLEX,4300,10.76
111,WR,Tim Patrick,WR/FLEX,4200,12.70
112,WR,Darius Slayton,WR/FLEX,4200,10.14
113,WR,Laviska Shenault Jr.,WR/FLEX,4100,9.62


In [643]:
qb_arr = qbs.to_numpy()
wr_arr = wrs.to_numpy()
rb_arr = rbs.to_numpy()
te_arr = tes.to_numpy()
flex_arr =flex_no_id.to_numpy()
def_array = defs.to_numpy()

In [644]:
rb_arr

array([['RB', 'Dalvin Cook', 'RB/FLEX', 9400, 26.94],
       ['RB', 'Christian McCaffrey', 'RB/FLEX', 9200, 30.13],
       ['RB', 'Derrick Henry', 'RB/FLEX', 8700, 20.74],
       ['RB', 'Aaron Jones', 'RB/FLEX', 7600, 19.93],
       ['RB', 'James Robinson', 'RB/FLEX', 7500, 19.73],
       ['RB', 'Alvin Kamara', 'RB/FLEX', 7100, 23.65],
       ['RB', 'Austin Ekeler', 'RB/FLEX', 7000, 16.15],
       ['RB', 'Chris Carson', 'RB/FLEX', 6900, 17.36],
       ['RB', 'Antonio Gibson', 'RB/FLEX', 6700, 15.93],
       ['RB', 'Ezekiel Elliott', 'RB/FLEX', 6600, 16.12],
       ['RB', "D'Andre Swift", 'RB/FLEX', 6500, 14.51],
       ['RB', 'David Montgomery', 'RB/FLEX', 6500, 15.16],
       ['RB', 'Mike Davis', 'RB/FLEX', 6400, 13.69],
       ['RB', 'Alexander Mattison', 'RB/FLEX', 6400, 5.26],
       ['RB', 'Josh Jacobs', 'RB/FLEX', 6300, 16.43],
       ['RB', 'Raheem Mostert', 'RB/FLEX', 6200, 14.15],
       ['RB', 'Miles Sanders', 'RB/FLEX', 6200, 13.04],
       ['RB', 'Ronald Jones II', 'RB/FLEX

In [670]:
#Function that generates a random team of daily fantasy football players.... Salary And duplicates not considered

def team_generator(qb, wr, rb, flex, defense):
    team_list = []
    
    #random players from each array
    qb_rows = qb.shape[0]
    random_indices = np.random.choice(qb_rows, size=1, replace=False)
    qb_rows = qb[random_indices, :]
    
    wr_rows = wr.shape[0]
    random_indices = np.random.choice(wr_rows, size=3, replace=False)
    wr_rows = wr[random_indices, :]
    
    
    rb_rows = rb.shape[0]
    random_indices = np.random.choice(rb_rows, size=2, replace=False)
    rb_rows = rb[random_indices, :]
    
    flex_rows = flex.shape[0]
    random_indices = np.random.choice(flex_rows, size=1, replace=False)
    flex_rows = flex[random_indices, :]
    
    def_rows = defense.shape[0]
    random_indices = np.random.choice(def_rows, size=1, replace=False)
    def_rows = defense[random_indices, :]
    
    
    
    
    team_list.append(qb_rows)
    team_list.append(wr_rows)
    team_list.append(rb_rows)
    team_list.append(flex_rows)
    team_list.append(def_rows)
    unpack_team = list(chain.from_iterable(team_list))
    
    un_pack_pd = pd.DataFrame(unpack_team)
    
    
   
        
    
    #print(un_pack_pd[3].sum())
    
    return unpack_team

random_team = team_generator(qb_arr, wr_arr, rb_arr, flex_arr, def_array)
random_team

[array(['QB', 'Matt Ryan', 'QB', 5700, 18.64], dtype=object),
 array(['WR', 'T.Y. Hilton', 'WR/FLEX', 5100, 9.8], dtype=object),
 array(['WR', 'Tim Patrick', 'WR/FLEX', 4200, 12.7], dtype=object),
 array(['WR', 'Adam Thielen', 'WR/FLEX', 7000, 19.42], dtype=object),
 array(['RB', "D'Andre Swift", 'RB/FLEX', 6500, 14.51], dtype=object),
 array(['RB', 'Raheem Mostert', 'RB/FLEX', 6200, 14.15], dtype=object),
 array(['WR', 'Tim Patrick', 'WR/FLEX', 4200, 12.7], dtype=object),
 array(['DST', 'Texans ', 'DST', 2600, 4.08], dtype=object)]

In [671]:
df = pd.DataFrame(random_team)

df

,0,1,2,3,4
0,QB,Matt Ryan,QB,5700,18.64
1,WR,T.Y. Hilton,WR/FLEX,5100,9.80
2,WR,Tim Patrick,WR/FLEX,4200,12.70
3,WR,Adam Thielen,WR/FLEX,7000,19.42
4,RB,D'Andre Swift,RB/FLEX,6500,14.51
5,RB,Raheem Mostert,RB/FLEX,6200,14.15
6,WR,Tim Patrick,WR/FLEX,4200,12.70
7,DST,Texans,DST,2600,4.08


In [672]:
#function that generates 1000 different samples

def sample_generator(samples = 1000):

    sample_list = []
    
    for _ in range(samples):
        sample_list.append(team_generator(qb_arr, wr_arr, rb_arr, flex_arr, def_array))
        
    unpack_samples = list(chain.from_iterable(sample_list))
        
    return unpack_samples

df = sample_generator(5)

big_panda = pd.DataFrame(df)
big_panda

,0,1,2,3,4
0,QB,Teddy Bridgewater,QB,5800,19.05
1,WR,Jalen Reagor,WR/FLEX,4400,8.76
2,WR,Corey Davis,WR/FLEX,5700,16.91
3,WR,Tyreek Hill,WR/FLEX,8500,23.66
4,RB,Myles Gaskin,RB/FLEX,5600,14.32
5,RB,David Johnson,RB/FLEX,5200,11.81
6,RB,Salvon Ahmed,RB/FLEX,4800,10.73
7,DST,Seahawks,DST,3000,5.83
8,QB,Matthew Stafford,QB,5700,18.88
9,WR,T.Y. Hilton,WR/FLEX,5100,9.80


In [668]:
big_panda.columns = ['pos', 'name', 'fant_pos', 'salary', 'proj_point']

In [669]:
big_panda

,pos,name,fant_pos,salary,proj_point
0,QB,Matthew Stafford,QB,5700,18.88
1,WR,Jalen Reagor,WR/FLEX,4400,8.76
2,WR,Mike Williams,WR/FLEX,4700,10.94
3,WR,Justin Jefferson,WR/FLEX,7400,18.68
4,RB,Ezekiel Elliott,RB/FLEX,6600,16.12
5,RB,Tevin Coleman,RB/FLEX,4900,2.58
6,RB,J.D. McKissic,RB/FLEX,4900,10.41
7,DST,Bears,DST,3000,5.42
8,QB,Justin Herbert,QB,6800,24.32
9,WR,Travis Fulgham,WR/FLEX,4900,11.86
